콘텐츠 필터링 - 체류시간 + 즐겨찾기

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from googleapiclient.discovery import build
from google.oauth2 import service_account

# 파일 경로 및 데이터 로드
file_path = '/content/CA4Udata.xlsx'
df = pd.read_excel(file_path)

# 필요한 칼럼만 선택 (수정된 칼럼으로 변경)
columns_to_use = ['Name', 'Category', 'Summary-1', 'Summary-2', 'Introduction',
                  'Fee', 'Date', 'Time', 'Place', 'Qualification', 'Method', 'Period',
                  'Affilation-1', 'Affilation-2', 'FeePeriod', 'Membership']
data = df[columns_to_use].copy()

# 불용어 리스트 (필요 시 수정)
stopwords = ['중앙대', '중앙대학교', '동아리', 'PM', '명', '기수', '기존', '기존부원', '신입', '신입부원', '1학기', '00', '정기모임', '기준', '약', '2024', '부원', '여']

# 전처리 함수
def preprocess_text(text):
    text = str(text)  # 문자열로 변환
    for stopword in stopwords:
        text = text.replace(stopword, '')
    return text

# 각 칼럼에 전처리 적용
for col in data.columns[1:]:
    data[col] = data[col].apply(preprocess_text)

# TF-IDF 벡터화 및 토큰 추출
tfidf_matrix = {}
tokens = {}

for col in data.columns[1:]:  # 'Name'은 제외하고 벡터화
    vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")  # 한 글자 단위도 포함
    tfidf_matrix[col] = vectorizer.fit_transform(data[col])
    tokens[col] = vectorizer.get_feature_names_out()

# 코사인 유사도 계산
cosine_sim = {}

for col in data.columns[1:]:
    cosine_sim[col] = cosine_similarity(tfidf_matrix[col])

# 가중치 적용 (수정된 가중치)
weights = {
    'Category': 0.5,
    'Summary-1': 0.5,
    'Summary-2': 0.5,
    'Introduction': 0.5,
    'Fee': 0.3,
    'Date': 0.3,
    'Time': 0.3,
    'Place': 0.3,
    'Qualification': 0.3,
    'Method': 0.3,
    'Period': 0.3,
    'Affilation-1': 0.1,
    'Affilation-2': 0.1,
    'FeePeriod': 0.1,
    'Membership': 0.1
}

# 최종 유사도 계산
final_similarity = np.zeros_like(cosine_sim[data.columns[1]])

for col in data.columns[1:]:
    final_similarity += weights[col] * cosine_sim[col]

# 구글 애널리틱스 API 설정
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
SERVICE_ACCOUNT_FILE = 'path_to_your_service_account_key.json'
PROPERTY_ID = 'your_property_id'

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
analytics = build('analyticsdata', 'v1beta', credentials=credentials)

def get_page_duration(property_id, start_date='30daysAgo', end_date='today'):
    request = {
        'entity': f'properties/{property_id}',
        'dimensions': [{'name': 'pagePath'}],
        'metrics': [{'name': 'averageSessionDuration'}],
        'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
        'orderBys': [{'metric': {'metricName': 'averageSessionDuration'}, 'desc': True}]
    }

    response = analytics.properties().runReport(
        property=property_id,
        body=request
    ).execute()

    return response

def extract_top_pages(response, top_n=3):
    rows = response.get('rows', [])
    top_pages = []

    for row in rows[:top_n]:
        page_path = row['dimensionValues'][0]['value']
        average_duration = float(row['metricValues'][0]['value'])
        top_pages.append((page_path, average_duration))

    return top_pages

def get_club_from_page_path(page_path, data):
    for index, row in data.iterrows():
        if row.get('PagePath') == page_path:
            return row['Name']
    return None

def recommend_clubs_with_page_data(user_id, user_DB, top_pages, data, final_similarity, top_n=3):
    all_recommended_clubs = []

    # 사용자 즐겨찾기 기반 추천
    user_favorites = user_DB[user_id]['favorites']
    for favorite_club in user_favorites:
        if favorite_club in data['Name'].values:
            idx = data[data['Name'] == favorite_club].index[0]
            sim_scores = list(enumerate(final_similarity[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = [x for x in sim_scores if x[0] != idx]
            top_clubs = [data['Name'][i[0]] for i in sim_scores[:top_n]]
            all_recommended_clubs.extend(top_clubs)

    # 페이지 체류 시간 기반 추천
    for page_path, _ in top_pages:
        # 페이지 경로와 관련된 동아리 찾기
        club_name = get_club_from_page_path(page_path, data)
        if club_name:
            # 해당 동아리와 유사한 동아리 찾기
            club_idx = data[data['Name'] == club_name].index[0]
            sim_scores = list(enumerate(final_similarity[club_idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

            # 유사도가 높은 동아리 추천 (본인은 제외)
            top_similar_clubs = [data['Name'][i[0]] for i in sim_scores if i[0] != club_idx][:top_n]
            all_recommended_clubs.extend(top_similar_clubs)

    # 중복 동아리 제거 후 반환
    return list(set(all_recommended_clubs))

# 사용 예시
user_id = 1  # 사용자 ID

# 구글 애널리틱스 데이터 추출 및 처리
response = get_page_duration(PROPERTY_ID)
top_pages = extract_top_pages(response)

# 사용자 DB 설정 (더미 데이터)
user_DB = {
    1: {'favorites': ['Science Club', 'Literature Club']}
}

# 추천 동아리 제공
recommended_clubs = recommend_clubs_with_page_data(user_id, user_DB, top_pages, data, final_similarity)
print(f"사용자 {user_id}의 추천 동아리: {recommended_clubs}")


사용자 협업 필터링

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 1. 사용자 MBTI 성향 저장용 데이터베이스 (딕셔너리로 저장)
user_DB = {}

# 2. 가중치 설정 (각 성향 요소에 대한 중요도 조정)
weights = {
    '학술': 0.5, '문화예술': 0.5, '운동': 0.5, '취미': 0.5, '종교': 0.5, '봉사': 0.5,
    'E': 0.3, 'I': 0.3, 'S': 0.3, 'N': 0.3, 'T': 0.3, 'F': 0.3, 'J': 0.3, 'P': 0.3
}
# 제일 중요한거 0.5, 두 번째가 0.3, 제일 안 중요한 것은 0.1

# 3. MBTI 질문에 대한 답변을 기록하는 함수
def record_answer(user_id, question_number, answer):
    if user_id not in user_DB:
        user_DB[user_id] = {'학술': 0, '문화예술': 0, '운동': 0, '취미': 0, '종교': 0, '봉사': 0,
                            'E': 0, 'I': 0, 'S': 0, 'N': 0, 'T': 0, 'F': 0, 'J': 0, 'P': 0}

    if question_number == 1: # 세 개의 순차적인 랭크로 카테고리 리스트 받는다고 가정
        score = 1.5
        for category_index in answer:
            if category_index == '학술':
                user_DB[user_id]['학술'] += score
            elif category_index == '문화예술':
                user_DB[user_id]['문화예술'] += score
            elif category_index == '운동':
                user_DB[user_id]['운동'] += score
            elif category_index == '취미':
                user_DB[user_id]['취미'] += score
            elif category_index == '종교':
                user_DB[user_id]['종교'] += score
            elif category_index == '봉사':
                user_DB[user_id]['봉사'] += score
            score -= 0.5
    elif question_number == 2:
        if answer == 'E':
            user_DB[user_id]['E'] += 1
        elif answer == 'I':
            user_DB[user_id]['I'] += 1
    elif question_number == 3:
        if answer == 'S':
            user_DB[user_id]['S'] += 1
        elif answer == 'N':
            user_DB[user_id]['N'] += 1
    elif question_number == 4:
        if answer == 'T':
            user_DB[user_id]['T'] += 1
        elif answer == 'F':
            user_DB[user_id]['F'] += 1
    elif question_number == 5:
        if answer == 'J':
            user_DB[user_id]['J'] += 1
        elif answer == 'P':
            user_DB[user_id]['P'] += 1
    else:
        raise ValueError("Invalid question number")

# 4. 사용자 기반 협업 필터링 함수 (가중치 적용)
def user_based_collaborative_filtering(target_user_id, k=3):
    # 사용자들의 성향 벡터를 가중치를 적용하여 행렬로 변환
    user_ids = list(user_DB.keys())
    DB_matrix = np.array([[user_DB[user]['학술'] * weights['학술'], user_DB[user]['문화예술'] * weights['문화예술'],
                          user_DB[user]['운동'] * weights['운동'], user_DB[user]['취미'] * weights['취미'],
                          user_DB[user]['종교'] * weights['종교'], user_DB[user]['봉사'] * weights['봉사'],
                          user_DB[user]['E'] * weights['E'], user_DB[user]['I'] * weights['I'],
                          user_DB[user]['S'] * weights['S'], user_DB[user]['N'] * weights['N'],
                          user_DB[user]['T'] * weights['T'], user_DB[user]['F'] * weights['F'],
                          user_DB[user]['J'] * weights['J'], user_DB[user]['P'] * weights['P']]
                            for user in user_ids])

    # 유사도 계산 (코사인 유사도 사용)
    similarity_matrix = cosine_similarity(DB_matrix)

    # 타겟 사용자의 인덱스 찾기
    target_index = user_ids.index(target_user_id)

    # 유사도가 높은 상위 k명의 사용자 선택
    similar_indices = np.argsort(similarity_matrix[target_index])[::-1][1:k+1]
    similar_users = [user_ids[i] for i in similar_indices]

    return similar_users

# 5. 테스트 예시
# 사용자 1이 MBTI 질문에 답변하는 예시
record_answer(1, 1, {'학술', '운동', '취미'})
record_answer(1, 2, 'E')
record_answer(1, 3, 'N')
record_answer(1, 4, 'T')
record_answer(1, 5, 'J')

# 사용자 2- ISFP
record_answer(2, 1, {'문화예술', '운동', '봉사'})
record_answer(2, 2, 'I')
record_answer(2, 3, 'S')
record_answer(2, 4, 'F')
record_answer(2, 5, 'P')

# 사용자 3 - ENTJ
record_answer(3, 1, {'종교', '봉사', '학술'})
record_answer(3, 2, 'E')
record_answer(3, 3, 'N')
record_answer(3, 4, 'T')
record_answer(3, 5, 'J')

# 사용자 4 - ISFP
record_answer(4, 1, {'학술', '봉사', '취미'})
record_answer(4, 2, 'I')
record_answer(4, 3, 'S')
record_answer(4, 4, 'F')
record_answer(4, 5, 'P')

# 사용자 5 - ENFP
record_answer(5, 1, {'문화예술', '취미', '봉사'})
record_answer(5, 2, 'E')
record_answer(5, 3, 'N')
record_answer(5, 4, 'F')
record_answer(5, 5, 'P')

# 사용자 6 - ISTJ
record_answer(6, 1, {'학술', '운동', '종교'})
record_answer(6, 2, 'I')
record_answer(6, 3, 'S')
record_answer(6, 4, 'T')
record_answer(6, 5, 'J')

# 사용자 7 - INFJ
record_answer(7, 1, {'봉사', '문화예술', '운동'})
record_answer(7, 2, 'I')
record_answer(7, 3, 'N')
record_answer(7, 4, 'F')
record_answer(7, 5, 'J')

# 사용자 8 - ENTP
record_answer(8, 1, {'학술', '취미', '종교'})
record_answer(8, 2, 'E')
record_answer(8, 3, 'N')
record_answer(8, 4, 'T')
record_answer(8, 5, 'P')

# 사용자 9 - ISFP
record_answer(9, 1, {'운동', '문화예술', '봉사'})
record_answer(9, 2, 'I')
record_answer(9, 3, 'S')
record_answer(9, 4, 'F')
record_answer(9, 5, 'P')

# 사용자 10 - INTJ
record_answer(10, 1, {'취미', '학술', '종교'})
record_answer(10, 2, 'I')
record_answer(10, 3, 'N')
record_answer(10, 4, 'T')
record_answer(10, 5, 'J')

# 사용자 11 - ESFJ
record_answer(11, 1, {'문화예술', '봉사', '운동'})
record_answer(11, 2, 'E')
record_answer(11, 3, 'S')
record_answer(11, 4, 'F')
record_answer(11, 5, 'J')

# 사용자 12 - INTP
record_answer(12, 1, {'종교', '취미', '학술'})
record_answer(12, 2, 'I')
record_answer(12, 3, 'N')
record_answer(12, 4, 'T')
record_answer(12, 5, 'P')

# 사용자 13 - ENTJ
record_answer(13, 1, {'운동', '문화예술', '봉사'})
record_answer(13, 2, 'E')
record_answer(13, 3, 'N')
record_answer(13, 4, 'T')
record_answer(13, 5, 'J')

# 사용자 14 - ISFJ
record_answer(14, 1, {'학술', '종교', '취미'})
record_answer(14, 2, 'I')
record_answer(14, 3, 'S')
record_answer(14, 4, 'F')
record_answer(14, 5, 'J')

# 사용자 15 - ESFP
record_answer(15, 1, {'문화예술', '운동', '봉사'})
record_answer(15, 2, 'E')
record_answer(15, 3, 'S')
record_answer(15, 4, 'F')
record_answer(15, 5, 'P')

# 사용자 16 - INFP
record_answer(16, 1, {'취미', '학술', '종교'})
record_answer(16, 2, 'I')
record_answer(16, 3, 'N')
record_answer(16, 4, 'F')
record_answer(16, 5, 'P')

# 사용자 17 - ESTJ
record_answer(17, 1, {'운동', '문화예술', '봉사'})
record_answer(17, 2, 'E')
record_answer(17, 3, 'S')
record_answer(17, 4, 'T')
record_answer(17, 5, 'J')

# 사용자 18 - ENFJ
record_answer(18, 1, {'학술', '취미', '종교'})
record_answer(18, 2, 'E')
record_answer(18, 3, 'N')
record_answer(18, 4, 'F')
record_answer(18, 5, 'J')

# 사용자 19 - ISTP
record_answer(19, 1, {'문화예술', '운동', '봉사'})
record_answer(19, 2, 'I')
record_answer(19, 3, 'S')
record_answer(19, 4, 'T')
record_answer(19, 5, 'P')

# 사용자 20 - ESTP
record_answer(20, 1, {'종교', '학술', '취미'})
record_answer(20, 2, 'E')
record_answer(20, 3, 'S')
record_answer(20, 4, 'T')
record_answer(20, 5, 'P')

# 사용자 21 - ENFP
record_answer(21, 1, {'운동', '문화예술', '봉사'})
record_answer(21, 2, 'E')
record_answer(21, 3, 'N')
record_answer(21, 4, 'F')
record_answer(21, 5, 'P')

# 사용자 22 - ISFP
record_answer(22, 1, {'학술', '종교', '취미'})
record_answer(22, 2, 'I')
record_answer(22, 3, 'S')
record_answer(22, 4, 'F')
record_answer(22, 5, 'P')

# 사용자 23 - INTJ
record_answer(23, 1, {'문화예술', '운동', '봉사'})
record_answer(23, 2, 'I')
record_answer(23, 3, 'N')
record_answer(23, 4, 'T')
record_answer(23, 5, 'J')

# 사용자 24 - ESFJ
record_answer(24, 1, {'취미', '학술', '종교'})
record_answer(24, 2, 'E')
record_answer(24, 3, 'S')
record_answer(24, 4, 'F')
record_answer(24, 5, 'J')

# 사용자 25 - ENTP
record_answer(25, 1, {'운동', '문화예술', '봉사'})
record_answer(25, 2, 'E')
record_answer(25, 3, 'N')
record_answer(25, 4, 'T')
record_answer(25, 5, 'P')

# 사용자 26 - INFJ
record_answer(26, 1, {'학술', '종교', '취미'})
record_answer(26, 2, 'I')
record_answer(26, 3, 'N')
record_answer(26, 4, 'F')
record_answer(26, 5, 'J')

# 사용자 27 - ESTP
record_answer(27, 1, {'문화예술', '운동', '봉사'})
record_answer(27, 2, 'E')
record_answer(27, 3, 'S')
record_answer(27, 4, 'T')
record_answer(27, 5, 'P')

# 사용자 28 - ISFJ
record_answer(28, 1, {'취미', '학술', '종교'})
record_answer(28, 2, 'I')
record_answer(28, 3, 'S')
record_answer(28, 4, 'F')
record_answer(28, 5, 'J')

# 사용자 29 - ENFJ
record_answer(29, 1, {'운동', '문화예술', '봉사'})
record_answer(29, 2, 'E')
record_answer(29, 3, 'N')
record_answer(29, 4, 'F')
record_answer(29, 5, 'J')

# 사용자 30 - ISTP
record_answer(30, 1, {'학술', '종교', '취미'})
record_answer(30, 2, 'I')
record_answer(30, 3, 'S')
record_answer(30, 4, 'T')
record_answer(30, 5, 'P')

# 사용자 31 - ENFP
record_answer(31, 1, {'문화예술', '운동', '봉사'})
record_answer(31, 2, 'E')
record_answer(31, 3, 'N')
record_answer(31, 4, 'F')
record_answer(31, 5, 'P')

# 사용자 32 - ISFP
record_answer(32, 1, {'취미', '학술', '종교'})
record_answer(32, 2, 'I')
record_answer(32, 3, 'S')
record_answer(32, 4, 'F')
record_answer(32, 5, 'P')

# 사용자 33 - ENTJ
record_answer(33, 1, {'운동', '문화예술', '봉사'})
record_answer(33, 2, 'E')
record_answer(33, 3, 'N')
record_answer(33, 4, 'T')
record_answer(33, 5, 'J')

# 사용자 34 - ESFP
record_answer(34, 1, {'학술', '종교', '취미'})
record_answer(34, 2, 'E')
record_answer(34, 3, 'S')
record_answer(34, 4, 'F')
record_answer(34, 5, 'P')

# 사용자 35 - INTP
record_answer(35, 1, {'문화예술', '운동', '봉사'})
record_answer(35, 2, 'I')
record_answer(35, 3, 'N')
record_answer(35, 4, 'T')
record_answer(35, 5, 'P')

# 사용자 36 - ESFJ
record_answer(36, 1, {'취미', '학술', '종교'})
record_answer(36, 2, 'E')
record_answer(36, 3, 'S')
record_answer(36, 4, 'F')
record_answer(36, 5, 'J')

# 사용자 37 - ISTJ
record_answer(37, 1, {'운동', '문화예술', '봉사'})
record_answer(37, 2, 'I')
record_answer(37, 3, 'S')
record_answer(37, 4, 'T')
record_answer(37, 5, 'J')

# 사용자 38 - ENTP
record_answer(38, 1, {'학술', '종교', '취미'})
record_answer(38, 2, 'E')
record_answer(38, 3, 'N')
record_answer(38, 4, 'T')
record_answer(38, 5, 'P')

# 사용자 39 - INFJ
record_answer(39, 1, {'문화예술', '운동', '봉사'})
record_answer(39, 2, 'I')
record_answer(39, 3, 'N')
record_answer(39, 4, 'F')
record_answer(39, 5, 'J')

# 사용자 40 - ISFP
record_answer(40, 1, {'취미', '학술', '종교'})
record_answer(40, 2, 'I')
record_answer(40, 3, 'S')
record_answer(40, 4, 'F')
record_answer(40, 5, 'P')

# 사용자 41 - ENTJ
record_answer(41, 1, {'운동', '문화예술', '봉사'})
record_answer(41, 2, 'E')
record_answer(41, 3, 'N')
record_answer(41, 4, 'T')
record_answer(41, 5, 'J')

# 사용자 42 - ESFJ
record_answer(42, 1, {'학술', '종교', '취미'})
record_answer(42, 2, 'E')
record_answer(42, 3, 'S')
record_answer(42, 4, 'F')
record_answer(42, 5, 'J')

# 사용자 43 - INFP
record_answer(43, 1, {'문화예술', '운동', '봉사'})
record_answer(43, 2, 'I')
record_answer(43, 3, 'N')
record_answer(43, 4, 'F')
record_answer(43, 5, 'P')

# 사용자 44 - ISTP
record_answer(44, 1, {'취미', '학술', '종교'})
record_answer(44, 2, 'I')
record_answer(44, 3, 'S')
record_answer(44, 4, 'T')
record_answer(44, 5, 'P')

# 사용자 45 - ESFP
record_answer(45, 1, {'운동', '문화예술', '봉사'})
record_answer(45, 2, 'E')
record_answer(45, 3, 'S')
record_answer(45, 4, 'F')
record_answer(45, 5, 'P')

# 사용자 46 - INTP
record_answer(46, 1, {'학술', '종교', '취미'})
record_answer(46, 2, 'I')
record_answer(46, 3, 'N')
record_answer(46, 4, 'T')
record_answer(46, 5, 'P')

# 사용자 47 - ESTJ
record_answer(47, 1, {'문화예술', '운동', '봉사'})
record_answer(47, 2, 'E')
record_answer(47, 3, 'S')
record_answer(47, 4, 'T')
record_answer(47, 5, 'J')

# 사용자 48 - INFJ
record_answer(48, 1, {'취미', '학술', '종교'})
record_answer(48, 2, 'I')
record_answer(48, 3, 'N')
record_answer(48, 4, 'F')
record_answer(48, 5, 'J')

# 사용자 49 - ESTP
record_answer(49, 1, {'운동', '문화예술', '봉사'})
record_answer(49, 2, 'E')
record_answer(49, 3, 'S')
record_answer(49, 4, 'T')
record_answer(49, 5, 'P')

# 사용자 50 - ESFJ
record_answer(50, 1, {'학술', '종교', '취미'})
record_answer(50, 2, 'E')
record_answer(50, 3, 'S')
record_answer(50, 4, 'F')
record_answer(50, 5, 'J')

# 사용자 51 - INFP
record_answer(51, 1, {'취미', '학술', '봉사'})
record_answer(51, 2, 'I')
record_answer(51, 3, 'N')
record_answer(51, 4, 'F')
record_answer(51, 5, 'P')

# 사용자 52 - ESTJ
record_answer(52, 1, {'문화예술', '운동', '종교'})
record_answer(52, 2, 'E')
record_answer(52, 3, 'S')
record_answer(52, 4, 'T')
record_answer(52, 5, 'J')

# 사용자 53 - ISFP
record_answer(53, 1, {'종교', '학술', '취미'})
record_answer(53, 2, 'I')
record_answer(53, 3, 'S')
record_answer(53, 4, 'F')
record_answer(53, 5, 'P')

# 사용자 54 - ENFJ
record_answer(54, 1, {'봉사', '문화예술', '운동'})
record_answer(54, 2, 'E')
record_answer(54, 3, 'N')
record_answer(54, 4, 'F')
record_answer(54, 5, 'J')

# 사용자 55 - INTJ
record_answer(55, 1, {'취미', '종교', '학술'})
record_answer(55, 2, 'I')
record_answer(55, 3, 'N')
record_answer(55, 4, 'T')
record_answer(55, 5, 'J')

# 사용자 56 - ESFP
record_answer(56, 1, {'운동', '봉사', '문화예술'})
record_answer(56, 2, 'E')
record_answer(56, 3, 'S')
record_answer(56, 4, 'F')
record_answer(56, 5, 'P')

# 사용자 57 - INFJ
record_answer(57, 1, {'학술', '종교', '운동'})
record_answer(57, 2, 'I')
record_answer(57, 3, 'N')
record_answer(57, 4, 'F')
record_answer(57, 5, 'J')

# 사용자 58 - ENFP
record_answer(58, 1, {'문화예술', '취미', '봉사'})
record_answer(58, 2, 'E')
record_answer(58, 3, 'N')
record_answer(58, 4, 'F')
record_answer(58, 5, 'P')

# 사용자 59 - ISFJ
record_answer(59, 1, {'종교', '학술', '운동'})
record_answer(59, 2, 'I')
record_answer(59, 3, 'S')
record_answer(59, 4, 'F')
record_answer(59, 5, 'J')

# 사용자 60 - ISTP
record_answer(60, 1, {'취미', '봉사', '문화예술'})
record_answer(60, 2, 'I')
record_answer(60, 3, 'S')
record_answer(60, 4, 'T')
record_answer(60, 5, 'P')

# 사용자 61 - ENTP
record_answer(61, 1, {'학술', '운동', '취미'})
record_answer(61, 2, 'E')
record_answer(61, 3, 'N')
record_answer(61, 4, 'T')
record_answer(61, 5, 'P')

# 사용자 62 - ESTP
record_answer(62, 1, {'문화예술', '종교', '봉사'})
record_answer(62, 2, 'E')
record_answer(62, 3, 'S')
record_answer(62, 4, 'T')
record_answer(62, 5, 'P')

# 사용자 63 - INFP
record_answer(63, 1, {'취미', '운동', '학술'})
record_answer(63, 2, 'I')
record_answer(63, 3, 'N')
record_answer(63, 4, 'F')
record_answer(63, 5, 'P')

# 사용자 64 - ESFJ
record_answer(64, 1, {'봉사', '문화예술', '운동'})
record_answer(64, 2, 'E')
record_answer(64, 3, 'S')
record_answer(64, 4, 'F')
record_answer(64, 5, 'J')

# 사용자 65 - INTJ
record_answer(65, 1, {'종교', '취미', '학술'})
record_answer(65, 2, 'I')
record_answer(65, 3, 'N')
record_answer(65, 4, 'T')
record_answer(65, 5, 'J')

# 사용자 66 - ISTJ
record_answer(66, 1, {'학술', '운동', '종교'})
record_answer(66, 2, 'I')
record_answer(66, 3, 'S')
record_answer(66, 4, 'T')
record_answer(66, 5, 'J')

# 사용자 67 - ENFJ
record_answer(67, 1, {'취미', '봉사', '문화예술'})
record_answer(67, 2, 'E')
record_answer(67, 3, 'N')
record_answer(67, 4, 'F')
record_answer(67, 5, 'J')

# 사용자 68 - ISFP
record_answer(68, 1, {'종교', '학술', '운동'})
record_answer(68, 2, 'I')
record_answer(68, 3, 'S')
record_answer(68, 4, 'F')
record_answer(68, 5, 'P')

# 사용자 69 - ENTP
record_answer(69, 1, {'운동', '취미', '문화예술'})
record_answer(69, 2, 'E')
record_answer(69, 3, 'N')
record_answer(69, 4, 'T')
record_answer(69, 5, 'P')

# 사용자 70 - ESTJ
record_answer(70, 1, {'학술', '종교', '봉사'})
record_answer(70, 2, 'E')
record_answer(70, 3, 'S')
record_answer(70, 4, 'T')
record_answer(70, 5, 'J')

# 사용자 71 - ENFP
record_answer(71, 1, {'문화예술', '운동', '취미'})
record_answer(71, 2, 'E')
record_answer(71, 3, 'N')
record_answer(71, 4, 'F')
record_answer(71, 5, 'P')

# 사용자 72 - INTP
record_answer(72, 1, {'봉사', '학술', '종교'})
record_answer(72, 2, 'I')
record_answer(72, 3, 'N')
record_answer(72, 4, 'T')
record_answer(72, 5, 'P')

# 사용자 73 - ISFJ
record_answer(73, 1, {'취미', '운동', '문화예술'})
record_answer(73, 2, 'I')
record_answer(73, 3, 'S')
record_answer(73, 4, 'F')
record_answer(73, 5, 'J')

# 사용자 74 - ESFP
record_answer(74, 1, {'종교', '학술', '봉사'})
record_answer(74, 2, 'E')
record_answer(74, 3, 'S')
record_answer(74, 4, 'F')
record_answer(74, 5, 'P')

# 사용자 75 - INTJ
record_answer(75, 1, {'운동', '취미', '문화예술'})
record_answer(75, 2, 'I')
record_answer(75, 3, 'N')
record_answer(75, 4, 'T')
record_answer(75, 5, 'J')

# 사용자 76 - ENFJ
record_answer(76, 1, {'학술', '종교', '운동'})
record_answer(76, 2, 'E')
record_answer(76, 3, 'N')
record_answer(76, 4, 'F')
record_answer(76, 5, 'J')

# 사용자 77 - ISTP
record_answer(77, 1, {'취미', '봉사', '문화예술'})
record_answer(77, 2, 'I')
record_answer(77, 3, 'S')
record_answer(77, 4, 'T')
record_answer(77, 5, 'P')

# 사용자 78 - ESTP
record_answer(78, 1, {'운동', '학술', '종교'})
record_answer(78, 2, 'E')
record_answer(78, 3, 'S')
record_answer(78, 4, 'T')
record_answer(78, 5, 'P')

# 사용자 79 - ISFP
record_answer(79, 1, {'봉사', '문화예술', '취미'})
record_answer(79, 2, 'I')
record_answer(79, 3, 'S')
record_answer(79, 4, 'F')
record_answer(79, 5, 'P')

# 사용자 80 - INTJ
record_answer(80, 1, {'학술', '운동', '종교'})
record_answer(80, 2, 'I')
record_answer(80, 3, 'N')
record_answer(80, 4, 'T')
record_answer(80, 5, 'J')

# 사용자 81 - ENTP
record_answer(81, 1, {'취미', '봉사', '문화예술'})
record_answer(81, 2, 'E')
record_answer(81, 3, 'N')
record_answer(81, 4, 'T')
record_answer(81, 5, 'P')

# 사용자 82 - ESFJ
record_answer(82, 1, {'운동', '종교', '학술'})
record_answer(82, 2, 'E')
record_answer(82, 3, 'S')
record_answer(82, 4, 'F')
record_answer(82, 5, 'J')

# 사용자 83 - ISTJ
record_answer(83, 1, {'문화예술', '취미', '운동'})
record_answer(83, 2, 'I')
record_answer(83, 3, 'S')
record_answer(83, 4, 'T')
record_answer(83, 5, 'J')

# 사용자 84 - ENFP
record_answer(84, 1, {'학술', '봉사', '종교'})
record_answer(84, 2, 'E')
record_answer(84, 3, 'N')
record_answer(84, 4, 'F')
record_answer(84, 5, 'P')

# 사용자 85 - ISFJ
record_answer(85, 1, {'취미', '운동', '학술'})
record_answer(85, 2, 'I')
record_answer(85, 3, 'S')
record_answer(85, 4, 'F')
record_answer(85, 5, 'J')

# 사용자 86 - ESFP
record_answer(86, 1, {'봉사', '문화예술', '종교'})
record_answer(86, 2, 'E')
record_answer(86, 3, 'S')
record_answer(86, 4, 'F')
record_answer(86, 5, 'P')

# 사용자 87 - INFP
record_answer(87, 1, {'운동', '취미', '학술'})
record_answer(87, 2, 'I')
record_answer(87, 3, 'N')
record_answer(87, 4, 'F')
record_answer(87, 5, 'P')

# 사용자 88 - INTJ
record_answer(88, 1, {'종교', '봉사', '문화예술'})
record_answer(88, 2, 'I')
record_answer(88, 3, 'N')
record_answer(88, 4, 'T')
record_answer(88, 5, 'J')

# 사용자 89 - ENFJ
record_answer(89, 1, {'학술', '운동', '취미'})
record_answer(89, 2, 'E')
record_answer(89, 3, 'N')
record_answer(89, 4, 'F')
record_answer(89, 5, 'J')

# 사용자 90 - ISFP
record_answer(90, 1, {'취미', '종교', '학술'})
record_answer(90, 2, 'I')
record_answer(90, 3, 'S')
record_answer(90, 4, 'F')
record_answer(90, 5, 'P')

# 사용자 91 - ESTJ
record_answer(91, 1, {'문화예술', '운동', '봉사'})
record_answer(91, 2, 'E')
record_answer(91, 3, 'S')
record_answer(91, 4, 'T')
record_answer(91, 5, 'J')

# 사용자 92 - ENTP
record_answer(92, 1, {'종교', '학술', '취미'})
record_answer(92, 2, 'E')
record_answer(92, 3, 'N')
record_answer(92, 4, 'T')
record_answer(92, 5, 'P')

# 사용자 93 - ISFJ
record_answer(93, 1, {'운동', '봉사', '문화예술'})
record_answer(93, 2, 'I')
record_answer(93, 3, 'S')
record_answer(93, 4, 'F')
record_answer(93, 5, 'J')

# 사용자 94 - ENFP
record_answer(94, 1, {'학술', '종교', '취미'})
record_answer(94, 2, 'E')
record_answer(94, 3, 'N')
record_answer(94, 4, 'F')
record_answer(94, 5, 'P')

# 사용자 95 - INFP
record_answer(95, 1, {'봉사', '문화예술', '운동'})
record_answer(95, 2, 'I')
record_answer(95, 3, 'N')
record_answer(95, 4, 'F')
record_answer(95, 5, 'P')

# 사용자 96 - ESFJ
record_answer(96, 1, {'취미', '종교', '학술'})
record_answer(96, 2, 'E')
record_answer(96, 3, 'S')
record_answer(96, 4, 'F')
record_answer(96, 5, 'J')

# 사용자 97 - ESTP
record_answer(97, 1, {'운동', '봉사', '문화예술'})
record_answer(97, 2, 'E')
record_answer(97, 3, 'S')
record_answer(97, 4, 'T')
record_answer(97, 5, 'P')

# 사용자 98 - INTJ
record_answer(98, 1, {'학술', '종교', '취미'})
record_answer(98, 2, 'I')
record_answer(98, 3, 'N')
record_answer(98, 4, 'T')
record_answer(98, 5, 'J')

# 사용자 99 - ENFJ
record_answer(99, 1, {'봉사', '문화예술', '운동'})
record_answer(99, 2, 'E')
record_answer(99, 3, 'N')
record_answer(99, 4, 'F')
record_answer(99, 5, 'J')

# 사용자 100 - ISFP
record_answer(100, 1, {'취미', '학술', '종교'})
record_answer(100, 2, 'I')
record_answer(100, 3, 'S')
record_answer(100, 4, 'F')
record_answer(100, 5, 'P')

# 사용자 1과 유사한 사용자 추출
similar_users = user_based_collaborative_filtering(1, k=10)
print("사용자 1과 유사한 사용자:", similar_users)

# 유사한 사용자들의 즐겨찾기 정보를 모두 가져오는 함수
def get_favorites_from_similar_users(similar_users):
    all_favorites = []  # 모든 유사한 사용자의 즐겨찾기를 저장할 리스트

    for user_id in similar_users:
        if user_id in user_DB:  # 해당 사용자가 user_DB에 있는지 확인
            user_favorites = user_DB[user_id]['favorites']
            all_favorites.extend(user_favorites)  # 해당 사용자의 즐겨찾기를 리스트에 추가
        else:
            print(f"사용자 ID '{user_id}'를 찾을 수 없습니다.")

    return list(set(all_favorites))  # 중복된 즐겨찾기 제거 후 반환


# 유사한 사용자들의 즐겨찾기 가져오기
favorites_from_similar_users = get_favorites_from_similar_users(similar_users)
print(f"유사한 사용자들의 즐겨찾기 동아리/학회 목록: {favorites_from_similar_users}")


하이브리드 추천 시스템 (콘텐츠 0.5 + 사용자 협업 0.5)

In [ ]:
#하이브리드 추천 시스템
def hybrid_recommendation(user_id, user_DB, top_pages, data, final_similarity, similar_users, alpha=0.5):
    # 콘텐츠 기반 필터링 추천
    content_based_recommendations = recommend_clubs_with_page_data(user_id, user_DB, top_pages, data, final_similarity)

    # 협업 필터링 추천
    collaborative_recommendations = get_favorites_from_similar_users(similar_users)

    # 콘텐츠 기반 추천 점수
    content_scores = {club: 1.0 for club in content_based_recommendations}

    # 협업 필터링 추천 점수
    collab_scores = {club: 1.0 for club in collaborative_recommendations}

    # 하이브리드 점수 계산
    hybrid_scores = {}

    # 콘텐츠 기반 추천과 협업 필터링 추천을 결합
    for club in set(content_based_recommendations + collaborative_recommendations):
        content_score = content_scores.get(club, 0)
        collab_score = collab_scores.get(club, 0)
        hybrid_scores[club] = alpha * content_score + (1 - alpha) * collab_score

    # 하이브리드 점수로 상위 top_n 추천
    recommended_clubs = sorted(hybrid_scores, key=hybrid_scores.get, reverse=True)[:top_pages]

    return recommended_clubs


# 하이브리드 추천 실행
recommended_clubs = hybrid_recommendation(user_id, user_DB, top_pages, data, final_similarity, similar_users, alpha=0.5)
print(f"사용자 {user_id}의 추천 동아리: {recommended_clubs}")

필요한 DB : 사용자 설문조사, 사용자 즐겨찾기, 동아리/학회